Importing Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [54]:
address = 'chennai, TamilNadu'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

C:\Users\Anand\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate are 13.0801721, 80.2838331.


In [55]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 100000 # define radius
CLIENT_ID = "FG03S1PEXXF32IF3PFDDIOT2SK4MIT2ETW3RSYOBUUSVPO4M" 
CLIENT_SECRET = "JHYQYGCY2XH1QWDXB4VO1SNJT1GO0CCDILIQK24SHWP222ET" 
version = '20191223'
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    version,
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FG03S1PEXXF32IF3PFDDIOT2SK4MIT2ETW3RSYOBUUSVPO4M&client_secret=JHYQYGCY2XH1QWDXB4VO1SNJT1GO0CCDILIQK24SHWP222ET&v=20191223&ll=13.0801721,80.2838331&radius=100000&limit=200'

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e490d275fb726001ba75690'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Chennai',
  'headerFullLocation': 'Chennai',
  'headerLocationGranularity': 'city',
  'totalResults': 176,
  'suggestedBounds': {'ne': {'lat': 13.980173000000901,
    'lng': 81.20608194628707},
   'sw': {'lat': 12.1801711999991, 'lng': 79.36158425371292}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b7fba44f964a520073b30e3',
       'name': 'Sathyam Cinemas',
       'location': {'address': '8, Thiru-vi-ka Road',
        'crossStreet': 'Royapettah',
        'lat': 13.055507758237352,
        'lng': 80.25798594343934,
        'labeledLatLngs': [{'label': 'display',
   

In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON
print(SGnearby_venues.shape)
SGnearby_venues[:3]

(100, 22)


,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b7fba44f964a520073b30e3-0,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",4b7fba44f964a520073b30e3,"8, Thiru-vi-ka Road",IN,Chennai,India,Royapettah,...,"[{'label': 'display', 'lat': 13.05550775823735...",13.055508,80.257986,NaN,600014,Tamil Nadu,Sathyam Cinemas,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c712b75f52cb71358fbb3af-1,"[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",4c712b75f52cb71358fbb3af,Express Avenue,IN,Chennai,India,"Whites Road, Royapettah",...,"[{'label': 'display', 'lat': 13.05874571403469...",13.058746,80.264170,NaN,NaN,Tamil Nadu,Escape Cinemas,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b9e3d3df964a52043d436e3-2,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",4b9e3d3df964a52043d436e3,11 Greams Road,IN,Chennai,India,NaN,...,"[{'label': 'display', 'lat': 13.06037679034977...",13.060377,80.254402,NaN,NaN,Tamil Nadu,Fruit Shop On Greams Road,0,[],NaN


In [72]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = ['Neighborhood','Category', 'Latitude', 'Longitude']


SGnearby_venues.head()

,Neighborhood,Category,Latitude,Longitude
0,Sathyam Cinemas,Movie Theater,13.055508,80.257986
1,Escape Cinemas,Multiplex,13.058746,80.264170
2,Fruit Shop On Greams Road,Juice Bar,13.060377,80.254402
3,Shree Mithai,Indian Restaurant,13.072118,80.247865
4,gabbana.life,Men's Store,13.060753,80.250768


In [73]:
SGnearby_venues.to_csv("nei_data.csv",index=False)

In [42]:
# create map of Madurai place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=6,
        radius=10,
        popup=label,
        color='blue').add_to(map_sg)  
    
map_sg